In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
# pip install tensorflow ignore if already installed

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')

In [6]:
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
# Features and target
X = dataset.iloc[:, 2:12]
X  # drop CustomerId and Surname

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
RowNumber,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
3,502,France,Female,42,8,159660.80,3,1,0,113931.57
4,699,France,Female,39,1,0.00,2,0,0,93826.63
5,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9996,771,France,Male,39,5,0.00,2,1,0,96270.64
9997,516,France,Male,35,10,57369.61,1,1,1,101699.77
9998,709,France,Female,36,7,0.00,1,0,1,42085.58


In [12]:
# Features and target
Y = dataset.iloc[:, 12].values  # Churn column
Y

array([1, 0, 1, ..., 1, 1, 0], shape=(10000,))

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [13]:
# Pipeline: One-hot encode categorical and scale
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        transformers=[
            ('gender', OneHotEncoder(drop='first'), ['Gender']),
            ('geo', OneHotEncoder(drop='first'), ['Geography'])
        ],
        remainder='passthrough'
    )),
    ('scaler', StandardScaler())
])

In [14]:
#Standardize the features
X = pipeline.fit_transform(X)

In [15]:
#Spilt the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [16]:
# Build ANN
classifier = Sequential()
classifier.add(Dense(6, activation='relu', input_shape=(X_train.shape[1],)))
classifier.add(Dropout(0.1))
classifier.add(Dense(6, activation='relu'))
classifier.add(Dropout(0.1))
classifier.add(Dense(1, activation='sigmoid'))

C:\Users\vishw\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Train ANN
history = classifier.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, verbose=2)

Epoch 1/100
225/225 - 3s - 13ms/step - accuracy: 0.7376 - loss: 0.5860 - val_accuracy: 0.7950 - val_loss: 0.4805
Epoch 2/100
225/225 - 1s - 3ms/step - accuracy: 0.7969 - loss: 0.4791 - val_accuracy: 0.7962 - val_loss: 0.4492
Epoch 3/100
225/225 - 1s - 4ms/step - accuracy: 0.7997 - loss: 0.4619 - val_accuracy: 0.8000 - val_loss: 0.4358
Epoch 4/100
225/225 - 1s - 3ms/step - accuracy: 0.8011 - loss: 0.4548 - val_accuracy: 0.8012 - val_loss: 0.4274
Epoch 5/100
225/225 - 1s - 3ms/step - accuracy: 0.8039 - loss: 0.4429 - val_accuracy: 0.7987 - val_loss: 0.4211
Epoch 6/100
225/225 - 1s - 3ms/step - accuracy: 0.8036 - loss: 0.4396 - val_accuracy: 0.8025 - val_loss: 0.4170
Epoch 7/100
225/225 - 1s - 3ms/step - accuracy: 0.8053 - loss: 0.4388 - val_accuracy: 0.8050 - val_loss: 0.4138
Epoch 8/100
225/225 - 1s - 3ms/step - accuracy: 0.8049 - loss: 0.4343 - val_accuracy: 0.8087 - val_loss: 0.4110
Epoch 9/100
225/225 - 1s - 3ms/step - accuracy: 0.8082 - loss: 0.4308 - val_accuracy: 0.8100 - val_loss

In [19]:
# Predict
y_pred = (classifier.predict(X_test) > 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [20]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[0.32752857]
 [0.3487643 ]
 [0.22522898]
 [0.08598081]
 [0.07558867]]


In [21]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [22]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1545   50]
 [ 220  185]]


In [23]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.5 % of data was classified correctly
